<a href="https://colab.research.google.com/github/MakkiFourati/RecSys-Thesis/blob/main/H%26M_RS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install turicreate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.0 MB 19 kB/s 
     |████████████████████████████████| 322 kB 56.5 MB/s 
     |████████████████████████████████| 3.5 MB 27.0 MB/s 
     |████████████████████████████████| 3.6 MB 59.8 MB/s 
     |████████████████████████████████| 86.4 MB 77 kB/s 
     |████████████████████████████████| 18.3 MB 25.5 MB/s 
     |████████████████████████████████| 2.9 MB 51.3 MB/s 
     |████████████████████████████████| 449 kB 73.9 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 3.8 MB 47.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=459e6a49b00e30f5cbfcea9adcbf884c24825bdc14585dcfac797e6e2f62c6a9
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6

In [ ]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from datetime import date, datetime
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('drive/MyDrive/Thesis/data/transactions_train.csv')

In [ ]:
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [ ]:
df.t_dat = pd.to_datetime(df.t_dat)
df.dtypes

t_dat               datetime64[ns]
customer_id                 object
article_id                   int64
price                      float64
sales_channel_id             int64
dtype: object

In [ ]:
df_tmp = df[(df['t_dat']>=datetime(2020,8,16))]

In [ ]:
df_tmp

,t_dat,customer_id,article_id,price,sales_channel_id
30398232,2020-08-16,000fb6e772c5d0023892065e659963da90b1866035558e...,902999001,0.059305,2
30398233,2020-08-16,000fb6e772c5d0023892065e659963da90b1866035558e...,760961003,0.101678,2
30398234,2020-08-16,000fb6e772c5d0023892065e659963da90b1866035558e...,932243003,0.025407,2
30398235,2020-08-16,000fb6e772c5d0023892065e659963da90b1866035558e...,915529001,0.033881,2
30398236,2020-08-16,001ddeb8fb74fec5693116da83b488e05ee9a9e179f3fd...,706016053,0.031949,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [ ]:
users_ids = df_tmp['customer_id'].unique()
users_dic = {u:j for u,j in zip(users_ids,range(len(users_ids)))}

In [ ]:
items_ids = df_tmp['article_id'].unique()
items_dic = {i:j for i,j in zip(items_ids ,range(len(items_ids)))}

In [ ]:
df_tmp['customer_id'] = df_tmp['customer_id'].map(users_dic)
df_tmp['article_id'] = df_tmp['article_id'].map(items_dic)
df_tmp

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,t_dat,customer_id,article_id,price,sales_channel_id
30398232,2020-08-16,0,0,0.059305,2
30398233,2020-08-16,0,1,0.101678,2
30398234,2020-08-16,0,2,0.025407,2
30398235,2020-08-16,0,3,0.033881,2
30398236,2020-08-16,1,4,0.031949,2
...,...,...,...,...,...
31788319,2020-09-22,37489,27389,0.059305,2
31788320,2020-09-22,37489,7524,0.042356,2
31788321,2020-09-22,156153,30304,0.043203,1
31788322,2020-09-22,227803,6017,0.006763,1


In [ ]:
len(df_tmp['customer_id'].unique())

286083

In [ ]:
df_train = df_tmp[df_tmp['t_dat']<=datetime(2020,9,15)]
df_val = df_tmp[df_tmp['t_dat']>=datetime(2020,9,16)]

In [ ]:
df_train.shape

(1149781, 5)

In [ ]:
df_val.shape

(240311, 5)

In [ ]:
print(len(df_train['customer_id'].unique()))
print(len(df_val['customer_id'].unique()))

249511
68984


In [ ]:
#df=df.drop(axis=1,columns=['t_dat','price','sales_channel_id'])
#df

In [ ]:
# create an SFrame out of df
dfS = tc.SFrame(df_tmp.iloc[:,1:3])
#dfS_whole = tc.SFrame(df.iloc[:,1:3])
dfS_train = tc.SFrame(df_train.iloc[:,1:3])
dfS_val = tc.SFrame(df_val.iloc[:,1:3])
dfS_train

customer_id,article_id
0,0
0,1
0,2
0,3
1,4
1,5
1,6
1,7
2,8
2,9


# 1 Item similarity recommender

## 1-1 Cosine Similarity

In [ ]:
cosine = tc.item_similarity_recommender.create(dfS_train, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              similarity_type='cosine')
# recommend k products for a list of users (None corresponds to all users) 
recommendation = cosine.recommend(users=None, k=12)
# print the n first recommendations
recommendation.print_rows(30)

Preparing data set.

Data has 1149781 observations with 249511 users and 29694 items.

Data prepared in: 1.23101s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 246000/249511 queries. users per second: 2597.81

recommendations finished on 247000/249511 queries. users per second: 2597.81

recommendations finished on 248000/249511 queries. users per second: 2597.5

recommendations finished on 249000/249511 queries. users per second: 2597.61

+-------------+------------+----------------------+------+
| customer_id | article_id |        score         | rank |
+-------------+------------+----------------------+------+
|    13990    |   96816    | 0.02265251033446368  |  1   |
|    13990    |   99292    | 0.02029078848221723  |  2   |
|    13990    |   99020    | 0.016634660608628216 |  3   |
|    13990    |   99277    | 0.016199441517100614 |  4   |
|    13990    |   98262    | 0.013634345110724954 |  5   |
|    13990    |   64373    | 0.013183127431308521 |  6   |
|    13990    |   99901    | 0.011291921138763428 |  7   |
|    13990    |   98961    | 0.010768031372743495 |  8   |
|    13990    |   98572    | 0.009394526481628418 |  9   |
|    13990    |   81609    | 0.00925880319931928  |  10  |
|    13990    |   100773   | 0.008978885762831745 |  11  |
|    13990    |   63051    | 0.008935935357037713 |  12  |
|    444730   |   75913    | 0.03952847421169281  |  1   |
|    444730   |   98194    | 0.03466876596212387  |  2  

In [ ]:
recommendation.print_rows(100)

+-------------+------------+----------------------+------+
| customer_id | article_id |        score         | rank |
+-------------+------------+----------------------+------+
|    13990    |   96816    | 0.02265251033446368  |  1   |
|    13990    |   99292    | 0.02029078848221723  |  2   |
|    13990    |   99020    | 0.016634660608628216 |  3   |
|    13990    |   99277    | 0.016199441517100614 |  4   |
|    13990    |   98262    | 0.013634345110724954 |  5   |
|    13990    |   64373    | 0.013183127431308521 |  6   |
|    13990    |   99901    | 0.011291921138763428 |  7   |
|    13990    |   98961    | 0.010768031372743495 |  8   |
|    13990    |   98572    | 0.009394526481628418 |  9   |
|    13990    |   81609    | 0.00925880319931928  |  10  |
|    13990    |   100773   | 0.008978885762831745 |  11  |
|    13990    |   63051    | 0.008935935357037713 |  12  |
|    444730   |   75913    | 0.03952847421169281  |  1   |
|    444730   |   98194    | 0.03466876596212387  |  2  

In [ ]:
# evaluating the model
cos_eval = cosine.evaluate_precision_recall(dfS_val)

recommendations finished on 1000/68984 queries. users per second: 1904.4

recommendations finished on 2000/68984 queries. users per second: 1848.76

recommendations finished on 3000/68984 queries. users per second: 1850.98

recommendations finished on 4000/68984 queries. users per second: 1844.35

recommendations finished on 5000/68984 queries. users per second: 1855.41

...

recommendations finished on 64000/68984 queries. users per second: 1645.4

recommendations finished on 65000/68984 queries. users per second: 1647.75

recommendations finished on 66000/68984 queries. users per second: 1650.69

recommendations finished on 67000/68984 queries. users per second: 1652.76

recommendations finished on 68000/68984 queries. users per second: 1655.34

In [ ]:
cos_eval

{'precision_recall_by_user': Columns:
 	customer_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 1241712
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | customer_id | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |    13990    |   1    |    0.0    |  0.0   |   9   |
 |    13990    |   2    |    0.0    |  0.0   |   9   |
 |    13990    |   3    |    0.0    |  0.0   |   9   |
 |    13990    |   4    |    0.0    |  0.0   |   9   |
 |    13990    |   5    |    0.0    |  0.0   |   9   |
 |    13990    |   6    |    0.0    |  0.0   |   9   |
 |    13990    |   7    |    0.0    |  0.0   |   9   |
 |    13990    |   8    |    0.0    |  0.0   |   9   |
 |    13990    |   9    |    0.0    |  0.0   |   9   |
 |    13990    |   10   |    0.0    |  0.0   |   9   |
 +-------------+--------+-----------+--------+-------+
 [1241712 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use pr

## 1-2 Jaccard similarity

In [ ]:
jaccard = tc.item_similarity_recommender.create(dfS_train, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              similarity_type='jaccard')
# recommend k products for a list of users (None corresponds to all users) 
recommendation_jac = jaccard.recommend(users=None, k=12)

Preparing data set.

Data has 1149781 observations with 249511 users and 29694 items.

Data prepared in: 1.55789s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 245000/249511 queries. users per second: 2561.4

recommendations finished on 246000/249511 queries. users per second: 2562.06

recommendations finished on 247000/249511 queries. users per second: 2562.98

recommendations finished on 248000/249511 queries. users per second: 2563.85

recommendations finished on 249000/249511 queries. users per second: 2564.54

In [ ]:
# print the n first recommendations
recommendation_jac.print_rows(20)

+-------------+------------+-----------------------+------+
| customer_id | article_id |         score         | rank |
+-------------+------------+-----------------------+------+
|      0      |    4345    |  0.012626262272105497 |  1   |
|      0      |    380     |  0.011028170585632324 |  2   |
|      0      |    1558    |  0.00903783826267018  |  3   |
|      0      |    5470    |  0.00857757470187019  |  4   |
|      0      |    2593    |  0.007222105475033031 |  5   |
|      0      |     47     | 0.0063916549963109635 |  6   |
|      0      |    8115    |  0.006204664707183838 |  7   |
|      0      |    980     |  0.005417869371526381 |  8   |
|      0      |    2289    |  0.004896784529966467 |  9   |
|      0      |   23743    |  0.004854885970844942 |  10  |
|      0      |    4178    |  0.004799064467935001 |  11  |
|      0      |   24220    |  0.004688136717852424 |  12  |
|      1      |    699     |  0.013108611106872559 |  1   |
|      1      |    1691    |  0.01306970

In [ ]:
jac_eval = jaccard.evaluate_precision_recall(dfS_val)

recommendations finished on 1000/68984 queries. users per second: 1977.7

recommendations finished on 2000/68984 queries. users per second: 1951.75

recommendations finished on 3000/68984 queries. users per second: 1934.53

recommendations finished on 4000/68984 queries. users per second: 1937.04

recommendations finished on 5000/68984 queries. users per second: 1932.43

...

recommendations finished on 64000/68984 queries. users per second: 1730.44

recommendations finished on 65000/68984 queries. users per second: 1732.98

recommendations finished on 66000/68984 queries. users per second: 1735.07

recommendations finished on 67000/68984 queries. users per second: 1737.3

recommendations finished on 68000/68984 queries. users per second: 1738.23

In [ ]:
jac_eval

{'precision_recall_by_user': Columns:
 	customer_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 1241712
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | customer_id | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |      0      |   1    |    0.0    |  0.0   |   9   |
 |      0      |   2    |    0.0    |  0.0   |   9   |
 |      0      |   3    |    0.0    |  0.0   |   9   |
 |      0      |   4    |    0.0    |  0.0   |   9   |
 |      0      |   5    |    0.0    |  0.0   |   9   |
 |      0      |   6    |    0.0    |  0.0   |   9   |
 |      0      |   7    |    0.0    |  0.0   |   9   |
 |      0      |   8    |    0.0    |  0.0   |   9   |
 |      0      |   9    |    0.0    |  0.0   |   9   |
 |      0      |   10   |    0.0    |  0.0   |   9   |
 +-------------+--------+-----------+--------+-------+
 [1241712 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use pr

In [ ]:
#training on the whole train data
jaccard = tc.item_similarity_recommender.create(dfS, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              similarity_type='jaccard')
# recommend k products for a list of users (None corresponds to all users) 
recommendation_jac = jaccard.recommend(users=None, k=12)

Preparing data set.

Data has 1390092 observations with 286083 users and 31086 items.

Data prepared in: 1.1728s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 282000/286083 queries. users per second: 3589.42

recommendations finished on 283000/286083 queries. users per second: 3590.53

recommendations finished on 284000/286083 queries. users per second: 3591.7

recommendations finished on 285000/286083 queries. users per second: 3592.7

recommendations finished on 286000/286083 queries. users per second: 3591.34

# 2 Matrix Factorization

Latent dimensionality: K < 18609

## 2-1 Adagrad and SGD

In [ ]:
mf_grad = tc.ranking_factorization_recommender.create(df_train, 
                                              user_id='user_id', 
                                              item_id='item_id',
                                              num_factors=10,
                                              #max_iterations=40
                                              solver='adagrad')
# recommend k products for a list of users (None corresponds to all users) 
recommendation = mf_grad.recommend(users=None, k=12)

Preparing data set.

Data has 16380622 observations with 305945 users and 42373 items.

Data prepared in: 16.0679s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

recommendations finished on 301000/305945 queries. users per second: 1376.21

recommendations finished on 302000/305945 queries. users per second: 1376.23

recommendations finished on 303000/305945 queries. users per second: 1376.33

recommendations finished on 304000/305945 queries. users per second: 1376.4

recommendations finished on 305000/305945 queries. users per second: 1376.54

In [ ]:
# print the n first recommendations
recommendation.print_rows(100)

+---------+---------+---------------------+------+
| user_id | item_id |        score        | rank |
+---------+---------+---------------------+------+
|    0    |  19240  | 0.35698288679122925 |  1   |
|    0    |   3921  |  0.3095870316028595 |  2   |
|    0    |   3876  |  0.2810938358306885 |  3   |
|    0    |   6153  | 0.22873052954673767 |  4   |
|    0    |   1179  | 0.21884199976921082 |  5   |
|    0    |   2350  | 0.20630986988544464 |  6   |
|    0    |   6691  | 0.19016516208648682 |  7   |
|    0    |   1744  | 0.16090434789657593 |  8   |
|    0    |   7937  | 0.15766994655132294 |  9   |
|    0    |   1125  | 0.15361566841602325 |  10  |
|    0    |   7928  | 0.15068849921226501 |  11  |
|    0    |   6079  | 0.13560670614242554 |  12  |
|    1    |   896   |  0.2353052794933319 |  1   |
|    1    |   3559  |  0.2041281759738922 |  2   |
|    1    |   5371  | 0.18876758217811584 |  3   |
|    1    |   3431  | 0.17464987933635712 |  4   |
|    1    |   361   |  0.139697

In [ ]:
mfg_eval = mf_grad.evaluate_precision_recall(df_test)

In [ ]:
mfg_eval

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 1619658
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |    5    |   1    |    0.0    |  0.0   |   9   |
 |    5    |   2    |    0.0    |  0.0   |   9   |
 |    5    |   3    |    0.0    |  0.0   |   9   |
 |    5    |   4    |    0.0    |  0.0   |   9   |
 |    5    |   5    |    0.0    |  0.0   |   9   |
 |    5    |   6    |    0.0    |  0.0   |   9   |
 |    5    |   7    |    0.0    |  0.0   |   9   |
 |    5    |   8    |    0.0    |  0.0   |   9   |
 |    5    |   9    |    0.0    |  0.0   |   9   |
 |    5    |   10   |    0.0    |  0.0   |   9   |
 +---------+--------+-----------+--------+-------+
 [1619658 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and c

## 2-2 iALS

In [ ]:
mf_als = tc.ranking_factorization_recommender.create(dfS_train, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              num_factors=200,
                                              #regularization=0.0001,
                                              max_iterations=50,
                                              solver='ials')
# recommend k products for a list of users (None corresponds to all users) 


Preparing data set.

Data has 1149781 observations with 249511 users and 29694 items.

Data prepared in: 1.12s

Training ranking_factorization_recommender for recommendations.

+------------------------------+--------------------------------------------------+----------+

...

| 48      | 1h 30m       | 0.816525                  |

| 49      | 1h 32m       | 0.81652                   |

| FINAL   | 1h 32m       | 0.81652                   |

+---------+--------------+---------------------------+

Optimization Complete: Iteration limit reached.

In [ ]:
recommendation_als = mf_als.recommend(users=None, k=12)

recommendations finished on 1000/249511 queries. users per second: 974.9

recommendations finished on 2000/249511 queries. users per second: 942.046

recommendations finished on 3000/249511 queries. users per second: 948.573

recommendations finished on 4000/249511 queries. users per second: 951.284

recommendations finished on 5000/249511 queries. users per second: 955.96

...

recommendations finished on 245000/249511 queries. users per second: 872.366

recommendations finished on 246000/249511 queries. users per second: 872.367

recommendations finished on 247000/249511 queries. users per second: 872.386

recommendations finished on 248000/249511 queries. users per second: 872.471

recommendations finished on 249000/249511 queries. users per second: 872.692

In [ ]:
# print the n first recommendations
recommendation_als.print_rows(20)

+-------------+------------+----------------------+------+
| customer_id | article_id |        score         | rank |
+-------------+------------+----------------------+------+
|      0      |    304     | 0.053468815982341766 |  1   |
|      0      |     58     | 0.03685280680656433  |  2   |
|      0      |    254     | 0.03209333494305611  |  3   |
|      0      |    5470    | 0.02973403036594391  |  4   |
|      0      |    360     | 0.029155481606721878 |  5   |
|      0      |    109     | 0.02635830268263817  |  6   |
|      0      |    479     | 0.025852560997009277 |  7   |
|      0      |    122     | 0.023259928449988365 |  8   |
|      0      |    1698    | 0.023181816563010216 |  9   |
|      0      |    195     | 0.022834401577711105 |  10  |
|      0      |     96     | 0.02091541513800621  |  11  |
|      0      |    648     | 0.016916681081056595 |  12  |
|      1      |    254     | 0.011271383613348007 |  1   |
|      1      |    304     | 0.009975682944059372 |  2  

In [ ]:
mfa_eval = mf_als.evaluate_precision_recall(dfS_val)

recommendations finished on 1000/68984 queries. users per second: 575.262

recommendations finished on 2000/68984 queries. users per second: 561.783

recommendations finished on 3000/68984 queries. users per second: 564.354

recommendations finished on 4000/68984 queries. users per second: 569.102

recommendations finished on 5000/68984 queries. users per second: 569.497

...

recommendations finished on 64000/68984 queries. users per second: 565.009

recommendations finished on 65000/68984 queries. users per second: 565.256

recommendations finished on 66000/68984 queries. users per second: 565.523

recommendations finished on 67000/68984 queries. users per second: 565.859

recommendations finished on 68000/68984 queries. users per second: 566.134

In [ ]:
mfa_eval

{'precision_recall_by_user': Columns:
 	customer_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 1241712
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | customer_id | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |      0      |   1    |    0.0    |  0.0   |   9   |
 |      0      |   2    |    0.0    |  0.0   |   9   |
 |      0      |   3    |    0.0    |  0.0   |   9   |
 |      0      |   4    |    0.0    |  0.0   |   9   |
 |      0      |   5    |    0.0    |  0.0   |   9   |
 |      0      |   6    |    0.0    |  0.0   |   9   |
 |      0      |   7    |    0.0    |  0.0   |   9   |
 |      0      |   8    |    0.0    |  0.0   |   9   |
 |      0      |   9    |    0.0    |  0.0   |   9   |
 |      0      |   10   |    0.0    |  0.0   |   9   |
 +-------------+--------+-----------+--------+-------+
 [1241712 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use pr

# 5 Recommending to new customers

## 1 Popularity

In [ ]:
recommendation_jac

customer_id,article_id,score,rank
0,25624,0.011904761904761904,1
0,4345,0.010389608996255057,2
0,380,0.00900240455354963,3
0,1558,0.0072163428579057965,4
0,5470,0.006177309013548351,5
0,2593,0.0057019562948317755,6
0,8115,0.005425226120721726,7
0,47,0.005372862021128337,8
0,2289,0.0049786510921659925,9
0,24220,0.0049484968185424805,10


In [ ]:
df_rec = recommendation_jac.to_dataframe()
df_rec

,customer_id,article_id,score,rank
0,0,25624,0.011905,1
1,0,4345,0.010390,2
2,0,380,0.009002,3
3,0,1558,0.007216,4
4,0,5470,0.006177,5
...,...,...,...,...
3432991,286082,7253,0.013158,8
3432992,286082,14771,0.012865,9
3432993,286082,8158,0.012821,10
3432994,286082,12983,0.012821,11


In [ ]:
df_rec = df_rec.loc[:,'customer_id':	'article_id']
df_rec

,customer_id,article_id
0,0,25624
1,0,4345
2,0,380
3,0,1558
4,0,5470
...,...,...
3432991,286082,7253
3432992,286082,14771
3432993,286082,8158
3432994,286082,12983


In [ ]:
inv_users_dic = {v: k for k, v in users_dic.items()}
inv_items_dic = {v: k for k, v in items_dic.items()}

In [ ]:
df_rec_t = df_rec.copy()
df_rec_t['article_id'] = df_rec_t['article_id'].apply(lambda x: '0'+str(inv_items_dic[x]))
df_rec_t['customer_id'] = df_rec_t['customer_id'].apply(lambda x: str(inv_users_dic[x]))
df_rec_t

,customer_id,article_id
0,000fb6e772c5d0023892065e659963da90b1866035558e...,0905660002
1,000fb6e772c5d0023892065e659963da90b1866035558e...,0903049003
2,000fb6e772c5d0023892065e659963da90b1866035558e...,0884319005
3,000fb6e772c5d0023892065e659963da90b1866035558e...,0927922001
4,000fb6e772c5d0023892065e659963da90b1866035558e...,0915529003
...,...,...
3432991,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,0769880001
3432992,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,0805000007
3432993,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,0770855001
3432994,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,0622710001


In [ ]:
df_rec_f = df_rec_t.groupby(by = ['customer_id'], as_index = False).agg({'article_id': ' '.join})

In [ ]:
df_rec_f.rename(columns={'article_id':'prediction'},inplace=True)

In [ ]:
df_rec_f

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0779781015 0858856005 0716670009 0568601007 05...
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321011 0805000001 0805000007 0805000008 07...
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0448509001 0448509018 0719530014 0714790020 09...
3,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,0568601007 0889652002 0903326006 0730863043 07...
4,0000b2f1829e23b24feec422ef13df3ccedaedc85368e6...,0914441004 0807244041 0706016003 0706016001 07...
...,...,...
286078,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,0737994021 0813051009 0630525039 0885970001 03...
286079,ffff61677073258d461e043cc9ed4ed97be5617a920640...,0889392001 0794567002 0889392002 0656067007 08...
286080,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0804992013 0740922001 0791587010 0854663005 07...
286081,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0762846027 0762846026 0889460003 0762846008 07...


In [ ]:
df_a = df_tmp.groupby('article_id',as_index=False).count().iloc[:,:2]
df_a.rename(columns={'t_dat': 'count'},inplace=True)
df_best = df_a.sort_values(by=['count'], ascending= False).iloc[:12]
df_best

,article_id,count
304,304,3471
1698,1698,2928
254,254,2911
360,360,2799
58,58,2788
5470,5470,2687
195,195,2418
109,109,2402
122,122,2239
56,56,2165


In [ ]:
df_best['article_id'] = df_best['article_id'].map(inv_items_dic)

In [ ]:
best = df_best['article_id'].values
best_articles = ' '.join('0' + str(x) for x in best)
best_articles

'0751471001 0918292001 0706016001 0915526001 0751471043 0915529003 0448509014 0896152002 0916468003 0863595006 0850917001 0898694001'

In [ ]:
df_cust = pd.read_csv('drive/MyDrive/Thesis/data/customers.csv')

In [ ]:
c = df_cust['customer_id'].to_list()
c_rec = df_rec_f['customer_id'].to_list()
new_customers = list(set(c) - set(c_rec))

In [ ]:
len(new_customers)

1085897

In [ ]:
df_new = pd.DataFrame({'customer_id' : new_customers})
df_new

,customer_id
0,b018518500e1bdba7c00e0c71e5136a42cdbb8e87c1aa0...
1,9725fea0542581b104a1bbc3fe8f68896fb67e8dc3512b...
2,2978dbdc38dd7ade1153537ca38976c766d2ebefabae54...
3,dd123e290fadf6a72d0674c0096a47497e21d687a15dee...
4,c9d73d8646459d3b10c4c6d6440f271050c57811c76cb1...
...,...
1085892,b710c067ade7d0e50b33c13dc3a331b98c8108427cf6c0...
1085893,e1152d8635b9e46781528635cdafba6bd2c0bdd72c48ef...
1085894,e71277406206205d5c8ac3b2f61d18527a41514fde51dc...
1085895,27cda611cd1e6aace8ceb47333fb6f523966eba02d063c...


In [ ]:
df_new['prediction'] = best_articles
df_new

,customer_id,prediction
0,b018518500e1bdba7c00e0c71e5136a42cdbb8e87c1aa0...,0751471001 0918292001 0706016001 0915526001 07...
1,9725fea0542581b104a1bbc3fe8f68896fb67e8dc3512b...,0751471001 0918292001 0706016001 0915526001 07...
2,2978dbdc38dd7ade1153537ca38976c766d2ebefabae54...,0751471001 0918292001 0706016001 0915526001 07...
3,dd123e290fadf6a72d0674c0096a47497e21d687a15dee...,0751471001 0918292001 0706016001 0915526001 07...
4,c9d73d8646459d3b10c4c6d6440f271050c57811c76cb1...,0751471001 0918292001 0706016001 0915526001 07...
...,...,...
1085892,b710c067ade7d0e50b33c13dc3a331b98c8108427cf6c0...,0751471001 0918292001 0706016001 0915526001 07...
1085893,e1152d8635b9e46781528635cdafba6bd2c0bdd72c48ef...,0751471001 0918292001 0706016001 0915526001 07...
1085894,e71277406206205d5c8ac3b2f61d18527a41514fde51dc...,0751471001 0918292001 0706016001 0915526001 07...
1085895,27cda611cd1e6aace8ceb47333fb6f523966eba02d063c...,0751471001 0918292001 0706016001 0915526001 07...


In [ ]:
submission = pd.concat([df_new,df_rec_f],axis=0)
submission

,customer_id,prediction
0,b018518500e1bdba7c00e0c71e5136a42cdbb8e87c1aa0...,0751471001 0918292001 0706016001 0915526001 07...
1,9725fea0542581b104a1bbc3fe8f68896fb67e8dc3512b...,0751471001 0918292001 0706016001 0915526001 07...
2,2978dbdc38dd7ade1153537ca38976c766d2ebefabae54...,0751471001 0918292001 0706016001 0915526001 07...
3,dd123e290fadf6a72d0674c0096a47497e21d687a15dee...,0751471001 0918292001 0706016001 0915526001 07...
4,c9d73d8646459d3b10c4c6d6440f271050c57811c76cb1...,0751471001 0918292001 0706016001 0915526001 07...
...,...,...
286078,ffff12aa623c69eae8959d673f1f12ad0194ad760d77fd...,0737994021 0813051009 0630525039 0885970001 03...
286079,ffff61677073258d461e043cc9ed4ed97be5617a920640...,0889392001 0794567002 0889392002 0656067007 08...
286080,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0804992013 0740922001 0791587010 0854663005 07...
286081,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0762846027 0762846026 0889460003 0762846008 07...


In [ ]:
submission.dtypes

customer_id    object
prediction     object
dtype: object

In [ ]:
len(submission['customer_id'].unique())

1371980

In [ ]:
submission.to_csv('my_submission.csv',index=False)